# Find Bell inequalities using linear programming

This notebooks demonstrates the locality test for a behavior $p$. We restrict the dimensions to two outputs $\Delta = 2$
and two inputs $m = 2$ at each of the two parties.

### Local example
First we will consider a local example. We take a probability distribution that is totally independent of the inputs, i.e.
all possible outcomes have the same probability $p(ab|xy) = \frac{1}{4}$.

In [23]:
import itertools
import numpy as np
# define inputs and outputs
inputs = [0,1]
outputs = [-1,1]
dim = (len(outputs)**2) * (len(inputs)**2)

# define probabilities
def prob(a,b,x,y):
    return 1/4

# define behavior
p = []
for x,y in itertools.product(inputs,inputs):
    for a,b in itertools.product(outputs,outputs):
        p.append(prob(a,b,x,y))
p = np.array(p)
assert len(p) == dim

We have now defined the behavior as a vector in $\mathbb{R}^{\Delta^2 \cdot m^2}$. Now we have to define the deterministic
behaviors, which are defined as:

$$ d_\lambda(ab|xy) = 1 \quad \text{if } a=a_x \text{ and } b=b_y \quad\text{; otherwise } 0$$

Hence we create a list of all possible local hidden variables (there are $\Delta^{2m}$) possibilities. For
each hidden variable, we iterate over the possible input/output combinations and set the entries to one, where the outputs
match to the ones predicted by the hidden variable. This looks a bit complex and there might be a better way to do it.
Tell me if you know it.

In [24]:
# create a list of all hidden variables
lhvs = itertools.product(outputs, repeat=2*len(inputs))
# list of all deterministic behaviors
deterministics = []
for lhv in lhvs:
    d = np.zeros(dim)
    counter = 0
    # iterate over the possible input and output combinations
    for x,y in itertools.product(inputs,inputs):
        for a,b in itertools.product(outputs,outputs):
            # check if the actual output is matching to the LHV output -> set entry in d_lambda to one if true
            if lhv[x] == a and lhv[y+len(inputs)] == b:
                d[counter] = 1.0
            counter += 1
    deterministics.append(d)


The dual form of the linear problem had the following form:

$$\text{maximize } S = s\cdot p - S_l \text{ such that}$$ <br>
$$ s \cdot d_\lambda - S_l \leq 0 \quad \forall \lambda = 1,..., \Delta^{2m} $$ <br>
$$ s \cdot p - S_l \leq 1 $$

However the *SciPy* solver only allows a dot product as the objective function. We therefore define an *extended behavior*
and *extended deterministic behaviors*, which are the usual behaviors with a $-1$ appended as last entry:

$$ \tilde{p} = (p, -1) \quad \text{and} \quad \tilde{d_\lambda} = (d_\lambda, -1) $$

Additionally the *SciPy* solver can only solve minimization problem. Thus we rewrite the problem as the following:

$$\text{minimize } -S = -\tilde{s}\cdot \tilde{p} \text{ such that}$$ <br>
$$ \tilde{s} \cdot \tilde{d_\lambda}  \leq 0 \quad \forall \lambda = 1,..., \Delta^{2m} $$ <br>
$$ \tilde{s} \cdot \tilde{p} \leq 1 $$

where $\tilde{s} = (s, S_l)$

In [25]:
#update the behavior
p = np.append(p, [-1.0])
# update the deterministic behaviors
for i in range(dim):
    deterministics[i] = np.append(deterministics[i],[-1.0])

The objective function and the inequalities need to be expressed in a specific  form, that the solver can handle them.
For more details you can look in the documentation of *SciPy*.

In [26]:
# objective function
obj = -p
# left hand side of inequalities
lhs_ineq = np.copy(deterministics)
lhs_ineq = np.append(lhs_ineq, [p], axis=0)


# right hand side of the inequalities
rhs_ineq = np.zeros(dim+1)
rhs_ineq[-1] = 1.0

# run the optimizer
from scipy.optimize import linprog
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq)
print(opt.message)

Optimization terminated successfully.


We can obtain the optimal values from the result of the optimizer. If the behavior is local it should fulfill the
inequality:

$$ S = s \cdot p - S_l \leq 0$$

In [27]:
S = np.dot(p,opt.x)
print('S = {}'.format(S))

S = 1.3259798814502233e-10
CHSH = 0.0


As we can see, the inequality $S = s \cdot p - S_l \leq 0$ is fulfilled (up to numerical uncertainty).
Thus the behavior we have tested is local, i.e. $p \in \mathcal{L}$.

# Non local behavior
Before we have used a local behavior, to show that the linear program gives the right solution for such a behavior.
Now we will redefine the behavior to be a non-local one. Therefore we will use the famous example of PR-box.

In [28]:
# probability for PR box
def prob(a,b,x,y):
    s = x*y
    if a == -1: a=0
    if b == -1: b=0
    t = (a+b) % 2
    return 1/2 * int(s==t)
# redefine the local behavior
p = []
for x,y in itertools.product(inputs,inputs):
    for a,b in itertools.product(outputs,outputs):
        p.append(prob(a,b,x,y))
p = np.array(p)
p = np.append(p, [-1.0])

Ready to run the optimization.

In [29]:
# objective function
obj = -p
# left hand side of inequalities
lhs_ineq = np.copy(deterministics)
lhs_ineq = np.append(lhs_ineq, [p], axis=0)

# right hand side of the inequalities
rhs_ineq = np.zeros(dim+1)
rhs_ineq[-1] = 1.0

# run the optimizer
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq)
print(opt.message)

print('S = {}'.format(np.dot(opt.x,p)))

# check the CHSH inequality
chsh = 0
for a,b in itertools.product(outputs,outputs):
    chsh += a*b*prob(a,b,0,0)
    chsh += a*b*prob(a,b,0,1)
    chsh += a*b*prob(a,b,1,0)
    chsh -= a*b*prob(a,b,1,1)
print('CHSH = {}'.format(chsh))

Optimization terminated successfully.
S = 0.9999999992615258
CHSH = 4.0
